# Mistral for MLflow

In [ ]:
# !pip install -q requirements.txt

In [2]:
class S3_provider():
    """
    Класс для взаимодействия с хранилищем S3.

    Этот класс предоставляет методы для загрузки файлов из S3 хранилища. Он используется для загрузки и
    хранения моделей и данных, необходимых для работы сервиса.
    """
    
    def __init__(self):
        """
        Инициализация провайдера S3.

        Настраивает соединение с хранилищем S3, используя заданные параметры подключения.
        """
        # Работа с облачными сервисами
        import s3fs
        import boto3
        from botocore.client import Config
    
        # Настройки MinIO
        minio_access_key  = "minio_access_key"
        minio_secret_key  = "minio_secret_key"
        minio_endpoint    = "minio_endpoint"
        minio_bucket_name = "minio_bucket_name"

        self.s3 = boto3.resource('s3',
                            endpoint_url=minio_endpoint,
                            aws_access_key_id='minio_access_key',
                            aws_secret_access_key='minio_secret_key',
                            config=Config(signature_version='s3v4'),
                            region_name='us-east-1')

        self.bucket_name = 'prod-aiplatform-data'
        self.bucket = self.s3.Bucket(self.bucket_name)

        self.s3 = s3fs.S3FileSystem(anon=False, 
                            key=minio_access_key, 
                            secret=minio_secret_key, 
                            client_kwargs={"endpoint_url": minio_endpoint},
                            use_ssl=False)


    def download_from_s3(self, s3_folder: str, local_folder: str) -> str:
        """
        Загрузка файлов из S3 хранилища в локальную директорию.

        Description:
            Метод автоматически загружает все файлы из указанной папки в S3 хранилище в локальную директорию.
            Если локальная директория не существует, она будет создана вместе с необходимыми поддиректориями.
            Процесс загрузки логируется, предоставляя информацию о статусе загрузки каждого файла.
            В случае возникновения ошибки в процессе загрузки, метод логирует ошибку и возвращает `None`.
        Args:
            s3_folder (str): Путь к папке в S3 хранилище. Указывается от корня бакета.
            local_folder (str): Путь к локальной папке для сохранения файлов.
        Returns:
            str or None: Возвращает путь к локальной директории, куда были загружены файлы, если процесс завершился
                         успешно. Возвращает `None`, если в процессе загрузки произошла ошибка.
        Exceptions:
            Логирует исключения, связанные с ошибками доступа к S3 или невозможностью создать локальные директории.
        """
        import os
        import logging
        
        logging.basicConfig(
            level=logging.INFO,
            format="%(asctime)s - [%(levelname)s]: %(message)s",
            handlers=[
                logging.handlers.RotatingFileHandler(
                    filename="log.log",
                    mode="a",
                    maxBytes=1024,
                    backupCount=1,
                    encoding=None,
                    delay=0),
                logging.StreamHandler()
                ]
              )

        if not os.path.exists(local_folder):
            try:
                for obj in self.bucket.objects.filter(Prefix=s3_folder):
                    # Формирование пути для сохранения файла локально
                    local_path = os.path.join(local_folder, os.path.basename(obj.key))

                    # Создание локальной папки, если она не существует
                    os.makedirs(os.path.dirname(local_path), exist_ok=True)

                    # Загрузка файла из S3 в локальную папку
                    self.bucket.download_file(obj.key, local_path)

                logging.info(f"Файлы успешно загружены из S3 в {local_path}")
            except Exception as e:
                logging.info(f"Ошибка при загрузке файла из S3: {str(e)}")
                return None
        return local_folder
    
def load_context(self, context):    
        """
        Загрузка моделей из S3 хранилища
        """
        import logging
        import s3fs
        from threading import Thread    

        # Создания экземпляра для взаимодействия с хранилищем S3
        self.s3_provider = S3_provider()
        self.is_model_ready = False
        thread = Thread(target=self.threaded_function)
        thread.start() 

In [ ]:
class MistralBot(mlflow.pyfunc.PythonModel):
    """
    Класс для управления чат-диалогом с помощью LLM.

    Description: Управляет взаимодействием между пользователем и моделью LLM, генерирует подсказки и ответы.
    """
    
    def threaded_function(self):
        """
        Загружает и инициализирует модель для генерации текста с использованием Hugging Face Transformers и langchain_community.llms.
        """
        # Импорт библиотек для работы с AWS S3, многопоточности, HuggingFace и токенизации
        import os                                                                      # Взаимодействие с операционной системой
        from botocore.client import Config                                             # Настройка клиента AWS S3
        from threading import Thread                                                   # Управление многопоточностью
        from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline  # Пайплайн для интеграции с langchain_community
        from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline         # Библиотека transformers для работы с моделями и токенизаторами

        print('Loading Mistral Bot model...')

        new_model_path = self.s3_provider.download_from_s3(s3_folder = "prod/mistral_7b_v0.2", local_folder = 'model')
        new_model_path = './model'
        
        # Загрузка токенизатора для предварительно обученной модели LLM
        self.tokenizer = AutoTokenizer.from_pretrained(new_model_path)
        
        # Загрузка предварительно обученной модели LLM с переносом на GPU для ускорения обработки
        self.model = AutoModelForCausalLM.from_pretrained(new_model_path, device_map="auto").to(device=self.device)  
        
        pipeline = pipeline(
                            "text-generation",
                               model=self.model,
                               tokenizer=self.tokenizer,
                               use_cache=True,
                               max_new_tokens=1200,
                               do_sample=True,
                               top_k=1,
                               temperature = 0.3,
                               num_return_sequences=1,
                               eos_token_id=self.tokenizer.eos_token_id,
                               pad_token_id=self.tokenizer.eos_token_id,
                        )
        
        self.pipeline = HuggingFacePipeline(pipeline=pipeline)
        
        self.is_model_ready = True
        print('Модель загрузилась')
        
    def load_context(self, context):
        """
        Загрузка моделей из S3 хранилища
        """
        import logging
        import s3fs
        from threading import Thread
        
        logging.basicConfig(level=logging.INFO,
                            format="%(asctime)s - [%(levelname)s]: %(message)s",
                            handlers=[
                                logging.handlers.RotatingFileHandler(
                                    filename="log.log", 
                                    mode="a",
                                    maxBytes=5120,
                                    backupCount=1,
                                    encoding=None,
                                    delay=0),
                                logging.StreamHandler()
                               ]
                              )
        
        # Создания экземпляра для взаимодействия с хранилищем S3
        self.s3_provider = S3_provider()
        self.device = 'cuda'
        self.gen_kwargs = {"bos_token_id": 1,
                           "do_sample": True,
                           "eos_token_id": 2,
                           "max_new_tokens": 1024, #1536
                           "no_repeat_ngram_size": 15,
                           "pad_token_id": 0,
                           "repetition_penalty": 1.1,
                           "temperature": 0.4,
                           "top_k": 40,
                           "top_p": 0.9}
        
        self.is_model_ready = False
        thread = Thread(target=self.threaded_function)
        thread.start()        

    def predict(self, context, model_input: pd.DataFrame) -> str:
        """
        Description:
            Генерация ответа на запрос пользователя с учетом подсказки.
        Args:
            model_input (pd.DataFrame): DataFrame содержащий id, query, promt.
            user_id (int): id пользователя.
            query (str): Текст запроса, который необходимо обработать.
            promt (str): Подсказка для LLM модели.
        Returns:
            str: Сгенерированный ответ модели.
        """
        import base64
        import pandas as pd
        import logging
        from langchain.chains import LLMChain

        if not self.is_model_ready:
            return {'status': 'model not ready'}

        # Предобработка DataFrame
        row = model_input.iloc[0]

        user_id = row['id']
        query   = row['query']
        prompt  = base64.b64decode(row['prompt']).decode("utf-8")

        try:
            if isinstance(model_input.gen_kwargs[0], dict):
                self.gen_kwargs = model_input.gen_kwargs[0]

                # Комбинирование запроса и подсказки
                inputs = self.tokenizer(prompt, return_tensors='pt')
                outputs = self.model.generate(inputs.input_ids.to(self.device), **self.gen_kwargs)
                generated_text = self.tokenizer.batch_decode(outputs[:, inputs.input_ids.shape[1]:], skip_special_tokens=True)[0]
        except:
            pass

        try:
            if model_input.gen_kwargs[0] == 'pipeline':
                llm_chain = LLMChain(prompt=prompt, llm=self.pipeline)
                generated_text = llm_chain.invoke(query)
        except:
            pass

        if generated_text:
            res = (pd.DataFrame({'user_id':[user_id], 'assistent_answer': [base64.b64encode(generated_text.strip().encode("utf-8")).decode("utf-8")]}).to_json()

        logging.info(f'\n___________________________________\n"id": {user_id}\n___________________________________\n"query": {query}\n___________________________________\n"prompt": {prompt}\n___________________________________\n"resp": {generated_text.strip()}\n___________________________________\n')
        return res

---
### Local test

In [4]:
import pandas as pd

# Создаем DataFrame с указанными полями и текстом
data = {
    "id": 611,
    "query": ["Как защититься от DDoS атак?"],
    "prompt": [" "]
}

responce = {
    "query_answer": ["Ответ будет в виде текста"]
}

input_df = pd.DataFrame(data)

output = pd.DataFrame(responce)

In [14]:
# Установка URI для MLflow трекинг сервера
mlflow.set_tracking_uri("http://mlflow")

# # Создание эксперимента
# mlflow.create_experiment('domain_nlu_retriever')

In [ ]:
# Начало MLflow эксперимента
with mlflow.start_run(experiment_id=11):
        mlflow.pyfunc.log_model(
            artifact_path='mistral_bot',
            python_model=VicunaBot(),
            signature=mlflow.models.signature.infer_signature(input_df, output),
            artifacts={"log": './log.log'},
            registered_model_name='mistralbotv1')

### Web Test

In [1]:
import base64
import pandas as pd
import requests

In [23]:
import requests

promt = """ """

dat = {'id':21,
       'query': 'Твоя задача ответить на следующий вопрос: Как зашититься от DDoS атак?',
       'prompt': [base64.b64encode(promt.encode("utf-8")).decode("utf-8")],
       'gen_kwargs': ['defaults']
      }

df_tmp = pd.DataFrame(dat)

model_url = 'https://your_url'

# Отправка POST-запроса с данными на предсказание
response = requests.post(model_url, json={'dataframe_records' : df_tmp.to_dict(orient='records')})

# Получение результата и дешифровка данных
check_text = response.json()['predictions'][0]['assistent_answer']

In [ ]:
print(base64.b64decode((check_text)).decode("utf-8"))